In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

from Project.Database import Db

In [2]:
year1_hour, meta1_hour  = Db.load_data(year=1, hourly=True, meta=True)
year2_hour, meta2_hour = Db.load_data(year=2, hourly=True, meta=True)

In [12]:
year1_hour['Timestamp'].dt.month

0       7
1       7
2       7
3       7
4       7
       ..
8756    6
8757    6
8758    6
8759    6
8760    7
Name: Timestamp, Length: 8761, dtype: int64

In [14]:
production = []
consumption = []

for df in [year1_hour, year2_hour]:
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(unit="h", arg=(df["Timestamp"].dt.month)) #.str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[1]).astype(int)
    df["Season"] = "summer"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([3,4,5])), "Season"] = "spring"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([9,10,11])), "Season"] = "fall"
    df.loc[(pd.DatetimeIndex(df["Timestamp"]).month.isin([1,2,12])), "Season"] = "winter"
    production.append(meta1_hour.loc[(meta1_hour['Parameter'] == "Power_Electrical") & (
    meta1_hour["Description"].str.contains("produced"))].index.tolist())

In [16]:
consumption_condition = (lambda self: ((self['Parameter'] == "Power_Electrical") | (self['Parameter'] == "Power_Thermal")) & (
    self["Description"].str.contains("power consumption" or "used")))

production_condition = (lambda self: (self['Parameter'] == "Power_Electrical") & (
    self["Description"].str.contains("produced")))

production_year1 = meta1_hour.loc[production_condition, meta1_hour.index].tolist()

consumption_year1 = meta1_hour.loc[consumption_condition, meta1_hour.index].tolist()

production_year2 = meta2_hour.loc[production_condition, meta2_hour.index].tolist()

consumption_year2 = meta2_hour.loc[consumption_condition, meta2_hour.index].tolist()

KeyError: "None of [Index(['DHW_ClothesWasherColdFlow', 'DHW_ClothesWasherHotFlow',\n       'DHW_DishwasherHotFlow', 'DHW_HeatPumpWaterHeaterEnergyTotal',\n       'DHW_HeatPumpWaterHeaterPowerTotal', 'DHW_ManifoldColdFlow',\n       'DHW_ManifoldHotFlow', 'DHW_MixValveColdFlow',\n       'DHW_RoomTempBasementHPWH', 'DHW_SHWGlycolTempIn',\n       ...\n       'HVAC_HeatPumpOutdoorUnitPower', 'HVAC_DehumidifierAirflow',\n       'HVAC_DehumidifierExitAirTemp', 'HVAC_DehumidifierInletAirTemp',\n       'HVAC_DehumidifierPower', 'Load_StatusRefrigerator',\n       'Load_StatusMicrowave', 'Load_StatusClothesWasher',\n       'Load_StatusDryerPowerTotal', 'Load_StatusHeatPumpWaterHeater'],\n      dtype='object', name='Unnamed: 0', length=360)] are in the [columns]"

In [ ]:
year1_hour["Production"] = year1_hour[production_year1].sum(1)
year1_hour["Consumption"] = year1_hour[consumption_year1].sum(1)

In [ ]:
year2_hour["Production"] = year2_hour[production_year2].sum(1)
year2_hour["Consumption"] = year2_hour[consumption_year2].sum(1)

In [ ]:
year1_hour["Season_order"] = [0 if Season == "spring" else 1 if Season == "summer" else 2 if Season == "fall" else 3 for Season in year1_hour["Season"]]
year1_hour.sort_values(by="Season_order", ascending=True, inplace=True)

In [ ]:
year2_hour["Season_order"] = [0 if Season == "spring" else 1 if Season == "summer" else 2 if Season == "fall" else 3 for Season in year2_hour["Season"]]
year2_hour.sort_values(by="Season_order", ascending=True, inplace=True)

In [ ]:
# Seasonal violin plots of production for both year 1 and 2
fig = px.violin(year1_hour, x="Season", y="Production", color="Season")
fig.write_html(Db.get_save_file_directory("Season_production_violin_year1.html"))

fig = px.violin(year2_hour, x="Season", y="Production", color="Season")
fig.write_html(Db.get_save_file_directory("Season_production_violin_year2.html"))

In [ ]:
# Seasonal violin plots of consumption for both year 1 and 2
fig = px.violin(year1_hour, x="Season", y="Consumption", color="Season")
fig.write_html(Db.get_save_file_directory("Season_consumption_violin_year1.html"))

fig = px.violin(year2_hour, x="Season", y="Consumption", color="Season")
fig.write_html(Db.get_save_file_directory("Season_consumption_violin_year2.html"))

In [ ]:
consumption_loads_condition = (lambda self: ((self['Parameter'] == "Power_Electrical") | (self['Parameter'] == "Power_Thermal")) & (
    self["Description"].str.contains("power consumption" or "used")) & (self["Subsystem"] == "Loads"))

consumption_loads_year1 = meta1_hour.loc[consumption_loads_condition, "Unnamed: 0"].tolist()

consumption_loads_year2 = meta2_hour.loc[consumption_loads_condition, "Unnamed: 0"].tolist()

year1_hour_season_aggr = year1_hour.groupby("Season")[consumption_loads_year1].sum(0)

In [ ]:
fig = px.bar(year1_hour_season_aggr, x=year1_hour_season_aggr.index, y=consumption_loads_year1)
fig.write_html(Db.get_save_file_directory("consumption_aggr_stacked_bar_season.html"))